Create `virtualenv` for this section with this:
```bash
virtualenv hw1-env -p python3.9
```
Install `mlflow`:
```bash
source hw1-env/bin/activate
pip install -r requirements.txt
```
Now select `hw1-env` as kernel in this Jupyter Notebook.

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?


In [1]:
!mlflow --version

mlflow, version 2.22.0


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```bash
python3 preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [4]:
!mkdir -p data/

!wget -qc https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet -O data/green_tripdata_2023-01.parquet
!wget -qc https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet -O data/green_tripdata_2023-02.parquet
!wget -qc https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet -O data/green_tripdata_2023-03.parquet

In [6]:
!python3 homework/preprocess_data.py --raw_data_path ./data --dest_path ./output

In [31]:
!ls output/
!ls output/ | wc -l 

dv.pkl	test.pkl  train.pkl  val.pkl
4


## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder [`homework`](homework). 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* 2
* 4
* 8
* 10

Start mlflow with this command:
```bash
mlflow server -h 0.0.0.0 -p 5000 --backend-store-uri sqlite:///mlflow.db
```

In [27]:
!python3 homework/train.py --data_path ./output

2025/05/24 09:25:52 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-train-exp' does not exist. Creating a new experiment.
5.431162180141208


`min_samples_split = 2`

## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

Full command:
```bash
mlflow server -h 0.0.0.0 -p 5000 --backend-store-uri sqlite:///mlflow.db --default-artifact-root artifacts
```


## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* 5.335
* 5.818
* 6.336


In [ ]:
!python3 homework/hpo.py

2025/05/24 09:40:23 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
🏃 View run delicate-kite-985 at: http://127.0.0.1:5000/#/experiments/2/runs/e7c9133a2cad4fa6aba3fef23f7323ce

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run delightful-doe-172 at: http://127.0.0.1:5000/#/experiments/2/runs/6215e2276fda45588c487af8d0ee0c15

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run colorful-fox-491 at: http://127.0.0.1:5000/#/experiments/2/runs/15db3aa92aaa4d20a83166042df011e7

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run rare-mule-614 at: http://127.0.0.1:5000/#/experiments/2/runs/9dcd104940ef41479f70ba24b48ade4a

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2                    

🏃 View run valuable-jay-849 at: http://127.0.0.1:5000/#/experiments/2/runs/d6d806ad1a82476689689d9fc9a

`RMSE`: 5.335 (5.335419588556921) 

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* 5.567
* 6.061
* 6.568

In [29]:
!python3 homework/register_model.py

2025/05/24 09:54:46 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
🏃 View run honorable-flea-994 at: http://127.0.0.1:5000/#/experiments/3/runs/c4ab4d1ec5e040af926ea4f9ca08f7ca
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run loud-hen-366 at: http://127.0.0.1:5000/#/experiments/3/runs/5851a33d18f64658b89036000a2a5af0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run likeable-fly-620 at: http://127.0.0.1:5000/#/experiments/3/runs/10d248f89e5a49cc8ac2e8cba52afd79
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run abrasive-mare-129 at: http://127.0.0.1:5000/#/experiments/3/runs/23d6c6878a374b14a332d532495aa9a6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run vaunted-cub-424 at: http://127.0.0.1:5000/#/experiments/3/runs/6ba246c67ea541919effdb2beb47d411
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
Successfully register

`RMSE`: 5.567 (5.567408012462019)